In [40]:
import pandas as pd
import numpy as np
import os
import nsepy
import wget
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from datetime import datetime, timedelta
from urllib.request import Request, urlopen
import shutil 

dfin = 0
start_year = 2012 #2008
for i in range(start_year,datetime.now().year):
    filen = "nifty" + str(i) + ".csv"
    if i == start_year:
        dfin = pd.read_csv(filen)
    else:
        dftemp = pd.read_csv(filen)
        dfin = dfin.append(dftemp)
    

# df2008 = pd.read_csv("nifty2008.csv")
# df2009 = pd.read_csv("nifty2009.csv")
# df2010 = pd.read_csv("nifty2010.csv")
# df2011 = pd.read_csv("nifty2011.csv")
# df2012 = pd.read_csv("nifty2012.csv")
# df2013 = pd.read_csv("nifty2013.csv")
# df2014 = pd.read_csv("nifty2014.csv")
# df2015 = pd.read_csv("nifty2015.csv")
# df2016 = pd.read_csv("nifty2016.csv")
# df2017 = pd.read_csv("nifty2017.csv")
# df2018 = pd.read_csv("nifty2018.csv")
# df2019 = pd.read_csv("nifty2019.csv")
# df2020 = pd.read_csv("nifty2020.csv")
# df2021 = pd.read_csv("nifty2021.csv")
# dfin = df2008.append(df2009)
# dfin = dfin.append(df2010)
# dfin = dfin.append(df2011)
# dfin = dfin.append(df2012)
# dfin = dfin.append(df2013)
# dfin = dfin.append(df2014)
# dfin = dfin.append(df2015)
# dfin = dfin.append(df2016)
# dfin = dfin.append(df2017)
# dfin = dfin.append(df2018)
# dfin = dfin.append(df2019)
# dfin = dfin.append(df2020)
dfin.drop(['Shares Traded','Open','Close','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin

,Date,High,Low
0,02-Jan-2012,4645.95,4588.05
1,03-Jan-2012,4773.10,4675.80
2,04-Jan-2012,4782.85,4728.85
3,05-Jan-2012,4779.80,4730.15
4,06-Jan-2012,4794.90,4686.85
...,...,...,...
247,24-Dec-2020,13771.75,13626.90
248,28-Dec-2020,13885.30,13811.55
249,29-Dec-2020,13967.60,13859.90
250,30-Dec-2020,13997.00,13864.95


In [43]:
from nsepy import get_history
from datetime import date
curr_year = datetime.now().year
dfcurr = pd.read_csv("nifty" + str(curr_year) + ".csv")
if len(dfcurr) == 0:
    start_curr_date = date(curr_year,1,1)
else:
    last_record_date = list(dfcurr.tail(1)['Date'])[0]
    last_record_date = datetime.strptime(last_record_date, "%d-%b-%Y")
    start_curr_date = date(last_record_date.year,last_record_date.month,last_record_date.day)
    start_curr_date = start_curr_date + timedelta(days=1)

nifty_50 = get_history(symbol="NIFTY 50",
                            start=start_curr_date,
                            end=date(curr_year,12,31),
                            index=True)
nifty_50['Date'] = nifty_50.index
nifty_50['Date'] = nifty_50['Date'].apply(lambda x: x.strftime("%d-%b-%Y"))
nifty_50 = nifty_50[['Date','Open','High','Low','Close','Volume','Turnover']]
nifty_50.rename(columns = {'Volume':'Shares Traded', 'Turnover':'Turnover (Rs. Cr)'}, inplace = True)
if len(nifty_50):
    nifty_50.tail()
else:
    print("No new records found starting",start_curr_date)


No new records found starting 2021-12-25


In [44]:
dfcurr = pd.read_csv("nifty" + str(curr_year) + ".csv")
dfcurr = dfcurr.append(nifty_50).reset_index(drop=True)
dfcurr.drop_duplicates(subset='Date', keep="first", inplace=True)
dfcurr.reset_index(drop=True, inplace=True)
dfcurr.to_csv("nifty" + str(curr_year) + ".csv", index=False)
nifty_50 = nifty_50[['Date','High','Low']]
dfin = dfin.append(dfcurr)
dfin.drop_duplicates(subset='Date', keep="first", inplace=True)
dfin.drop(['Shares Traded','Open','Close','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin.reset_index(inplace=True,drop=True)
dfin

,Date,High,Low
0,02-Jan-2012,4645.95,4588.05
1,03-Jan-2012,4773.10,4675.80
2,04-Jan-2012,4782.85,4728.85
3,05-Jan-2012,4779.80,4730.15
4,06-Jan-2012,4794.90,4686.85
...,...,...,...
2469,20-Dec-2021,16840.10,16410.20
2470,21-Dec-2021,16936.40,16688.25
2471,22-Dec-2021,16971.00,16819.50
2472,23-Dec-2021,17118.65,17015.55


In [45]:
# n=4 is quaterly compounding
# r=5 is 5% per year

def comp_amount(p, t, n=4, r=5):
    # t is in months
    t = t/12
    interest_amount = p * (pow((1 + (r / (100 * n))), n * t))
    return interest_amount

def comp_amount_df(r):
    return comp_amount(r['AllocationAmount'], r['CompoundMonth'])

# i = comp_amount(5000,12)
# i

In [47]:
df = dfin.copy()
df['Date'] = df['Date'].apply(lambda x : datetime.strptime(x,"%d-%b-%Y"))
df.loc[len(df.index)] = [datetime.combine(date.today(), datetime.min.time()),16960.25,16833.20] 
df.tail()

,Date,High,Low
2470,2021-12-21,16936.40,16688.25
2471,2021-12-22,16971.00,16819.50
2472,2021-12-23,17118.65,17015.55
2473,2021-12-24,17155.60,16909.60
2474,2021-12-27,16960.25,16833.20


In [48]:
def print_last_save(last_save):
    try:
        day_diff = (last_save[-1][2] - last_save[0][2]) + timedelta(days=1)
        print(f"Select Highest Price as {last_save[-1][0]} from {last_save[0][1]} on {last_save[-1][2]} -> {day_diff}")
    except Exception as e:
#         print(last_save)
        pass

In [49]:
temp_dict = {}
percentage_drop_expected = 5

In [50]:
select_price= list(df.iloc[:1]['High'])[0]
select_date = list(df.iloc[:1]['Date'])[0]
start_year = select_date.year
# find day higher than today or 4% down that today
last_save = []
for i, r in df.iterrows():
#     if i == 500: break
#     print(list(r), select_price, ((select_price - r['Low']) * 100 / select_price))
    if ((select_price - r['Low']) * 100 / select_price) >= percentage_drop_expected:
#         print_last_save(last_save)
        last_save = []
        low_price = r['Low']
        low_date = r['Date'] #.strftime("%Y-%m-%d")
        print(f'{percentage_drop_expected}% drop from {select_price} on {select_date} to {low_price} on {low_date}')
            
        temp_dict[i] = {'HighPrice' : select_price, 
                                    'HighPriceDate': select_date, 
                                    'LowPrice' : low_price,
                                    'LowPriceDate'   : low_date,
                                    }
        select_price = low_price
    elif r['High'] > select_price:
#         print(select_date, "high para")
        high_price = r['High']
        high_date = r['Date']
        last_save.append([high_price,select_price,high_date])
        select_price = high_price
        select_date = high_date
    else:
        pass

#         print(select_date,"no match")

#         select_price = r['Low']
#         select_date = r['Date'] #.strftime("%Y-%m-%d")
print_last_save(last_save)
# df.head(30)


5% drop from 5629.95 on 2012-02-22 00:00:00 to 5268.15 on 2012-02-27 00:00:00
5% drop from 5458.8 on 2012-02-29 00:00:00 to 5171.45 on 2012-03-07 00:00:00
5% drop from 5499.4 on 2012-03-14 00:00:00 to 5205.65 on 2012-03-22 00:00:00
5% drop from 5378.75 on 2012-04-03 00:00:00 to 5070.6 on 2012-05-04 00:00:00
5% drop from 5124.75 on 2012-05-07 00:00:00 to 4837.05 on 2012-05-16 00:00:00
5% drop from 5348.55 on 2012-07-10 00:00:00 to 5076.6 on 2012-07-25 00:00:00
5% drop from 5807.25 on 2012-10-04 00:00:00 to 4888.2 on 2012-10-05 00:00:00
5% drop from 6111.8 on 2013-01-29 00:00:00 to 5748.6 on 2013-02-26 00:00:00
5% drop from 5971.2 on 2013-03-11 00:00:00 to 5647.95 on 2013-03-21 00:00:00
5% drop from 6229.45 on 2013-05-20 00:00:00 to 5916.35 on 2013-06-03 00:00:00
5% drop from 5981.6 on 2013-06-04 00:00:00 to 5645.65 on 2013-06-20 00:00:00
5% drop from 6093.35 on 2013-07-23 00:00:00 to 5747.6 on 2013-07-30 00:00:00
5% drop from 5808.5 on 2013-08-01 00:00:00 to 5486.85 on 2013-08-07 00:00:

In [51]:
def sell_price_hit_df(row):
    return sell_price_hit(row["LowPriceDate"],row["SellPrice"])

def sell_price_hit(start_date, sell_price):
#     start_date = datetime.strptime(start_date,"%Y-%m-%d")
    dfw = df[(df["Date"] >= start_date) & (df["High"] >=  sell_price)].head(1)['Date']
    if len(dfw) == 0:
        return date(curr_year + 1,12,31)
    else:
        return list(dfw)[0]

# record = sell_price_hit("2008-01-16",6990.90)
# record

In [52]:
temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
temp_df.reset_index(inplace=True,drop=True)
fin_df = pd.DataFrame(temp_df)
fin_df = fin_df.sort_values(by=['LowPriceDate'], axis=0).reset_index(drop=True)
fin_df['PercentDrop'] =  (fin_df['LowPrice'] - fin_df['HighPrice'] ) * 100 / fin_df['LowPrice'] 
fin_df['SellPrice'] =  fin_df['LowPrice'] * 1.2
fin_df['SellDate'] = fin_df.apply(sell_price_hit_df,axis=1)
fin_df.head()

,HighPrice,HighPriceDate,LowPrice,LowPriceDate,PercentDrop,SellPrice,SellDate
0,5629.95,2012-02-22,5268.15,2012-02-27,-6.867686,6321.78,2013-11-01
1,5458.80,2012-02-29,5171.45,2012-03-07,-5.556469,6205.74,2013-05-20
2,5499.40,2012-03-14,5205.65,2012-03-22,-5.642907,6246.78,2013-10-24
3,5378.75,2012-04-03,5070.60,2012-05-04,-6.077190,6084.72,2013-01-21
4,5124.75,2012-05-07,4837.05,2012-05-16,-5.947840,5804.46,2012-10-04


In [53]:
current_etf_price = 182.27
current_nifty_price = 17003.75
invest_amount = 12000
increment_investment_each_year = 0.25

fin_df['Year'] = fin_df['LowPriceDate'].apply(lambda x: x.year)
fin_df['BeginYear'] = fin_df['Year'].diff(periods=-1)
fin_df.loc[fin_df.index[-1], 'BeginYear']= -1

etf_factor =  current_nifty_price / current_etf_price
fin_df['PriceCutOff'] = fin_df['HighPrice'] * (1-percentage_drop_expected/100)
fin_df['ETFPurchasePrice'] = fin_df['PriceCutOff'] / etf_factor

inv_factor = []
prev_price = 0
prev_factor = 1
prev_date = list(fin_df.head(1)['LowPriceDate'])[0]
for i, r in fin_df.iterrows():
#     print(prev_date.year, r['LowPriceDate'].year)
    diff = prev_date.year -  r['LowPriceDate'].year 
    if int(r['ETFPurchasePrice']) < int(prev_price) and diff == 0:
        prev_factor = prev_factor * (1 + increment_investment_each_year)
        inv_factor.append(prev_factor)
    else:
        prev_factor = 1
        inv_factor.append(prev_factor)
    prev_price = r['ETFPurchasePrice']
    prev_date = r['LowPriceDate']

fin_df['InvFactor'] = inv_factor
fin_df['AllocationAmount'] = (fin_df['Year'] - start_year) * increment_investment_each_year
fin_df['AllocationAmount'] = ( fin_df['AllocationAmount'] + 1 ) * invest_amount * fin_df['InvFactor']
fin_df.head()

,HighPrice,HighPriceDate,LowPrice,LowPriceDate,PercentDrop,SellPrice,SellDate,Year,BeginYear,PriceCutOff,ETFPurchasePrice,InvFactor,AllocationAmount
0,5629.95,2012-02-22,5268.15,2012-02-27,-6.867686,6321.78,2013-11-01,2012,0.0,5348.4525,57.332202,1.0000,12000.0
1,5458.80,2012-02-29,5171.45,2012-03-07,-5.556469,6205.74,2013-05-20,2012,0.0,5185.8600,55.589308,1.2500,15000.0
2,5499.40,2012-03-14,5205.65,2012-03-22,-5.642907,6246.78,2013-10-24,2012,0.0,5224.4300,56.002756,1.0000,12000.0
3,5378.75,2012-04-03,5070.60,2012-05-04,-6.077190,6084.72,2013-01-21,2012,0.0,5109.8125,54.774125,1.2500,15000.0
4,5124.75,2012-05-07,4837.05,2012-05-16,-5.947840,5804.46,2012-10-04,2012,0.0,4868.5125,52.187534,1.5625,18750.0


In [54]:
fin_df['ETFSharesPurchased'] = fin_df['AllocationAmount'] // fin_df['ETFPurchasePrice'] 
fin_df['CumETFSharesPurchased'] = fin_df['ETFSharesPurchased'].cumsum()
fin_df['InvestedAmount'] = fin_df['ETFSharesPurchased'] * fin_df['ETFPurchasePrice']
fin_df['CumInvestedAmount'] = fin_df['InvestedAmount'].cumsum()
# fin_df['SellPrice'] = fin_df['SellPrice'] / etf_factor
fin_df['SellPrice'] = current_etf_price
fin_df['ETFProfit'] = (fin_df['SellPrice'] * fin_df['ETFSharesPurchased'])- fin_df['InvestedAmount']
fin_df['ETFProfit'] = (current_nifty_price - fin_df['PriceCutOff']) * fin_df['ETFSharesPurchased'] / etf_factor
fin_df['CumETFProfit'] = fin_df['ETFProfit'].cumsum()
# fin_df['CompoundMonth'] = fin_df.apply(lambda x: (datetime.now().year - x['LowPriceDate'].date().year) * 12 + (datetime.now().month - x['LowPriceDate'].date().month),axis=1)
# fin_df['Compounding'] = fin_df.apply(comp_amount_df,axis=1)
# fin_df['CumCompounding'] = fin_df['Compounding'].cumsum()

fin_df.tail()

,HighPrice,HighPriceDate,LowPrice,LowPriceDate,PercentDrop,SellPrice,SellDate,Year,BeginYear,PriceCutOff,ETFPurchasePrice,InvFactor,AllocationAmount,ETFSharesPurchased,CumETFSharesPurchased,InvestedAmount,CumInvestedAmount,ETFProfit,CumETFProfit
67,15336.30,2021-03-12,14478.60,2021-03-18,-5.923915,182.27,2021-09-06,2021,0.0,14569.4850,156.176139,1.2500,48750.0,312.0,22354.0,48726.955504,2.043095e+06,8141.284496,2.031368e+06
68,14984.15,2021-04-08,14191.40,2021-04-19,-5.586130,182.27,2021-08-31,2021,0.0,14234.9425,152.590045,1.5625,60937.5,399.0,22753.0,60883.427763,2.103979e+06,11842.302237,2.043211e+06
69,18604.45,2021-10-19,17613.10,2021-10-29,-5.628481,182.27,2022-12-31,2021,0.0,17674.2275,189.457117,1.0000,39000.0,205.0,22958.0,38838.708903,2.142817e+06,-1473.358903,2.041737e+06
70,18210.15,2021-11-15,17280.45,2021-11-22,-5.380068,182.27,2022-12-31,2021,0.0,17299.6425,185.441790,1.2500,48750.0,262.0,23220.0,48585.749007,2.191403e+06,-831.009007,2.040906e+06
71,17639.50,2021-12-13,16410.20,2021-12-20,-7.491073,182.27,2022-12-31,2021,-1.0,16757.5250,179.630616,1.5625,60937.5,339.0,23559.0,60894.778723,2.252298e+06,894.751277,2.041801e+06


In [55]:
# # dfss = fin_df.sort_values(by=['SellDate'], axis=0).copy().reset_index(drop=True)
# dfss = fin_df.copy()
# tinv = []
# tpro = []
# for i,r in dfss.iterrows():
# #     print(list(r))
#     df_bal = fin_df[(fin_df["LowPriceDate"] <= r['LowPriceDate'])][['ETFProfit','InvestedAmount','SellDate']]
#     tot_profit, total_investamt = 0, 0
#     if len(df_bal) > 0:
#         total_investamt = sum(list(df_bal['InvestedAmount']))
#         df_prof = df_bal[(df_bal["SellDate"] <= r['LowPriceDate'])][['ETFProfit']]
#         tot_profit = sum(list(df_prof['ETFProfit']))
# #     print(total_investamt, tot_profit)
#     tinv.append(total_investamt)
#     tpro.append(tot_profit)
# #     if i == 5: break
# fin_df['Running_TotalInv'] = tinv
# fin_df['Running_Total_Profit'] = tpro

In [60]:
last_highest = last_save[-1][0] / etf_factor
buy_price = last_save[-1][0] * (1 - percentage_drop_expected/100) / etf_factor
print(f"{last_highest} ==> {buy_price}")
invest_amt_down = list(fin_df.tail(1)['AllocationAmount'])[0] * (1 + increment_investment_each_year)
invest_amt_up = (curr_year - start_year) * increment_investment_each_year * invest_amount
print(f"Invest Amount if lower ==> {invest_amt_down} & Quantity ==> {invest_amt_down // buy_price}")
print(f"Invest Amount if above ==> {invest_amt_up}")

183.89774091009335 ==> 174.7028538645887
Invest Amount if lower ==> 76171.875 & Quantity ==> 436.0
Invest Amount if above ==> 27000.0


In [58]:
fin_df.drop(["Year","HighPrice","HighPriceDate","LowPrice","PercentDrop","PriceCutOff"],axis=1,inplace=True, errors='ignore')
fin_df.drop(["CompoundMonth","Compounding","CumCompounding","SellPrice","SellDate","PrevETFPurchasePrice"],axis=1,inplace=True, errors='ignore')
fin_df.to_csv("nifty_buyondip_"+ str(percentage_drop_expected) +"percent.csv")
# dfin.to_csv("nifty_all.csv")

In [59]:
# https://www1.nseindia.com/live_market/dynaContent/live_watch/equities_stock_watch.htm?cat=N
# https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=NIFTY
# https://github.com/webclinic017/test/blob/d185cdab9cefdb89a50f350a2df7c45f17e9e2bc/nse/utils/utils.py
# from nsetools import Nse
# nse = Nse()
# q = nse.get_index_quote('NIFTY 50')
# q